# Header

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import os

exec(open("../header.py").read())

Header initialized


# Import

In [26]:
threshold = 40
threshold = str(threshold)

In [27]:
raw_train_data = pd.read_csv(processed_root("04-bow-added-features/feat_train_data-"+threshold+".csv"))
raw_val_data = pd.read_csv(processed_root("04-bow-added-features/feat_val_data-"+threshold+".csv"))
raw_test_data = pd.read_csv(processed_root("04-bow-added-features/feat_test_data-"+threshold+".csv"))

In [28]:
X_train = raw_train_data\
    .drop(['Unnamed: 0', 'poetry_text', 'poetry_author'], axis = 1)
X_val = raw_val_data\
    .drop(['Unnamed: 0', 'poetry_text', 'poetry_author'], axis = 1)
y_train = raw_train_data['poetry_author']
y_val = raw_val_data['poetry_author']

X_test = raw_test_data\
    .drop(['Unnamed: 0', 'poetry_text', 'poetry_author'], axis = 1)
y_test = raw_test_data['poetry_author']

In [29]:
len(X_train.columns)

1901

# Best model

In [30]:
model = DecisionTreeClassifier(max_depth = 35, class_weight = 'balanced')

# Validation scores

In [31]:
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=35, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [32]:
# No single feature accounting for things
np.max(model.feature_importances_)

0.1694533126522822

In [33]:
# Most important word
X_train.iloc[0:1,np.argmax(model.feature_importances_):np.argmax(model.feature_importances_)+1]

,ave_line_len
0,4.528571


In [34]:
y_pred = model.predict(X_val)

In [35]:
accuracy = sum(y_pred == y_val)/len(y_val)

In [36]:
author_accuracy = pd.DataFrame({'pred':y_pred,
              'true':y_val})\
    .assign(correct = lambda x:x.pred == x.true)\
    .groupby('true', as_index = False)\
    .mean()\
    .loc[:,'correct']\
    .mean()

In [37]:
print("Accuracy: {:.3f}, Author-weighted Accuracy: {:.3f}".format(accuracy, author_accuracy))

Accuracy: 0.480, Author-weighted Accuracy: 0.463


# Test scores

In [17]:
X_train_combined = pd.concat([X_train, X_val], axis = 0, ignore_index = True)
y_train_combined = pd.concat([y_train, y_val], axis = 0, ignore_index = True)

In [18]:
model.fit(X_train_combined, y_train_combined)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=35, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [19]:
# No single feature accounting for things
np.max(model.feature_importances_)

0.15895900461739298

In [20]:
# Most important word
X_train.iloc[0:1,np.argmax(model.feature_importances_):np.argmax(model.feature_importances_)+1]

,ave_line_len
0,4.528571


In [21]:
y_pred = model.predict(X_test)

In [22]:
accuracy = sum(y_pred == y_test)/len(y_test)

In [23]:
author_accuracy = pd.DataFrame({'pred':y_pred,
              'true':y_test})\
    .assign(correct = lambda x:x.pred == x.true)\
    .groupby('true', as_index = False)\
    .mean()\
    .loc[:,'correct']\
    .mean()

In [24]:
print("Accuracy: {:.3f}, Author-weighted Accuracy: {:.3f}".format(accuracy, author_accuracy))

Accuracy: 0.559, Author-weighted Accuracy: 0.526
